# DataRobot API example
### Read the csv dataset from the file system, splitting off a small percentage from the training set to use for scoring predictions.

In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score, log_loss, roc_curve, auc
from pprint import pprint
import os
import time
import requests
import datarobot as dr
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

source_data_file = 'data/flight_delay_data_50k.csv'
target = 'DELAY_INDICATOR'

# Read in data from .csv
df_full = pd.read_csv(source_data_file)

# Shuffle the rows
df_full = df_full.sample(frac=1, random_state=0)

# Split 90% for training, 10% for predictions
split = int(df_full.shape[0] * .1)
df_train = df_full[split:]
df_pred_y = df_full[:split].copy()  # has target column

# Drop the target column from the prediction dataset
df_pred = df_pred_y.drop(target, axis=1)

print('Full data shape:      ',df_full.shape)
print('Training data shape:  ',df_train.shape)
print('Prediction data shape:',df_pred.shape)

Full data shape:       (50000, 16)
Training data shape:   (45000, 16)
Prediction data shape: (5000, 15)


## Create a DataRobot project

In [2]:
# Establish client connection
dr.Client(token=os.environ['DATAROBOT_API_TOKEN'], endpoint='https://app.datarobot.com/api/v2')

## The 'Out of the Box Autopilot'
### If we wanted to run autopilot on a dataset and it's features as-is, then we would simply run project.start, which only needs the source data (eg, a csv), the target name, and a project name as parameters.  Project creation, file upload and target selection are all combined in Project.start method.  DataRobot will do the rest in terms of data typing and using informative features.
```
project.start(sourcedata=df, 
              target='DELAY_INDICATOR'
              project_name='Flight delay classification'
```
### Instead, we'll use a few steps to create a project, upload the data, and set the target.  After feature engineering, we can run full autopilot or zmanually train selected models.

#### Autopilot mode means that the modeling process will proceed completely automatically, including running recommended models, running at different sample sizes, and blending.

#### Manual mode means that DataRobot will populate a list of recommended models, but will not insert any of them into the queue. Manual mode lets you select which models to execute before starting the modeling process.



In [3]:
import time
t1 = time.time()
# Create a project: upload the data, set the target, give the project a name
# Blueprint processing and training will run asynchronously after performing the second EDA pass
project = dr.Project.create(sourcedata=df_train, 
                            project_name='Flight Delay API example')

print('project id:  ', project.id)
print('project name:', project.project_name)
print('project.get_status():\n- ', project.get_status())
print('- create done: %0.3f' % (time.time()-t1))

t1 = time.time()
# Setting the target starts autopilot, but we'll use manual mode to kick off autopilot in a later step.
project.set_target(target=target, 
                   mode='manual',
                   worker_count=4)
print('- set_target done: %0.3f' % (time.time()-t1))

project id:   5be0f3aefd2b980f3ceecb0b
project name: Flight Delay API example
project.get_status():
-  {'autopilot_done': False, 'stage_description': 'Ready to set target', 'stage': 'aim'}
- create done: 31.113
- set_target done: 33.093


## Working with features
###  We'll generate a new feature list to do some feature engineering given some dirty data and our domain knowledge of the business.  Start by first retrieving the features from the project object.

In [4]:
# Output the identified project features
features = project.get_features()
sorted([f.name for f in features])

['ACT_AC_TYPE',
 'ARRV_A15_IND',
 'AVOIDABLE_DELAY_IND',
 'CARRIER',
 'DAY_OF_WEEK',
 'DELAY_CODE',
 'DELAY_CODE_DESCRIPTION',
 'DELAY_INDICATOR',
 'DESTINATION',
 'D_O_W_NUM',
 'FLT_DATE',
 'FLT_DATE (Day of Week)',
 'FLT_NBR',
 'HOUR_OF_DPTR',
 'ORIGIN',
 'UNAVOIDABLE_DELAY_IND',
 'Unnamed: 0']

### We'll transform the flght number feature from numeric to categorical, and create a new feature list omitting features with target leakage, such as other flight delay flags.

In [5]:
# Convert flight number to a categorical variable
new_feature = project.create_type_transform_feature(name='FLT_NBR_CAT', 
                                      parent_name='FLT_NBR', 
                                      variable_type='categoricalInt')
new_feature

Feature(FLT_NBR_CAT)

In [6]:
# Create a new feature list from a subset of features to eliminate target leaks and use the categorical flight number
new_list = ['DELAY_INDICATOR', 'ORIGIN', 'DESTINATION', 'CARRIER', 
            'DAY_OF_WEEK', 'HOUR_OF_DPTR', 'ACT_AC_TYPE', new_feature.name]
featurelist = project.create_featurelist(name='my feature list', features=new_list)
featurelist

Featurelist(my feature list)

## Start Autopilot

### Option 1 - Full Autopilot mode

In [7]:
project.start_autopilot(featurelist.id)

# Block until complete
# print('Waiting to complete')
# project.wait_for_autopilot()

### Option 2 - Manual mode
#### Select specific models to run

In [7]:
# Get the blueprints DataRobot provides in the repository.
# The available blueprints are those appropriate given our dataset and target type. 
blueprints = project.get_blueprints()

# Select a few blueprints to train those models
selected_blueprints = []
ex = en = rl = True
for bp in blueprints:
    if 'eXtreme' in bp.model_type and ex:
        selected_blueprints.append(bp)
        ex = False
    if 'Elastic-Net' in bp.model_type and en:
        selected_blueprints.append(bp)
        en = False
    if 'Regularized' in bp.model_type and rl:
        selected_blueprints.append(bp)
        rl = False
    if len(selected_blueprints) > 2:
        break
selected_blueprints

[Blueprint(eXtreme Gradient Boosted Trees Classifier with Early Stopping),
 Blueprint(Elastic-Net Classifier (L1 / Binomial Deviance)),
 Blueprint(Regularized Logistic Regression (L2))]

In [13]:
model_jobs = []
for bp in selected_blueprints:
    try:
        model_job_id = project.train(bp, featurelist_id=featurelist.id,
                                     scoring_type='crossValidation')
        model_jobs.append(model_job_id)
    except Exception as e:
        print( bp.model_type,'\n', type(e))
print('model jobs:', model_jobs)

wait_secs = 10
while True:
    jobs = project.get_model_jobs()
    print('Jobs in queue: %s' % len(jobs))
    if len(jobs) == 0:
        print('- Done.')
        break
    else:
        pprint(jobs)
        time.sleep(wait_secs)

eXtreme Gradient Boosted Trees Classifier with Early Stopping 
 <class 'datarobot.errors.JobAlreadyRequested'>
Elastic-Net Classifier (L1 / Binomial Deviance) 
 <class 'datarobot.errors.JobAlreadyRequested'>
Regularized Logistic Regression (L2) 
 <class 'datarobot.errors.JobAlreadyRequested'>
model jobs: []
Jobs in queue: 0
- Done.


## Get models and blueprints from the leaderboard

### To get the best performing model from the leaderboard, pop it from the top of the models list.  Or if run via full autopilot, the recommended model is available

In [16]:
# Get the models, which are already ordered by rank from the leaderboard
models = project.get_models()

print('Number of models on the leaderboard: %s\n' % len(models))

# Get the best performing model (excluding the blenders, which are typically the top 4 models)
for model in models:
    if 'Blender' not in model.model_type:
        best_model = model
        break
print('Best model from the leaderboard:  \'%s\'' % best_model.model_type)

# Or get the recommended model, if available
try:
    recommendation = dr.ModelRecommendation.get(project.id)
    recommended_model = recommendation.get_model()
    print('Recommended model:\'%s\'' % recommended_model.model_type)
except Exception as e:
    print('No recommended model yet.  Either autopilot is still running or models were trained manually.')
# **Note for a deployment, you have to manually add the deployment and get the deployment ID from the UI.
#   Can't do this via the api at this time.

# Get the blueprint
blueprint_id = best_model.blueprint_id
blueprint = dr.models.Blueprint.get(project.id, blueprint_id)
print('\nBest model blueprint preprocessing steps:')
pprint(blueprint.processes)

# Get the model scoring metrics
print('\nBest model metrics:')
pprint(best_model.metrics)

Number of models on the leaderboard: 56

Best model from the leaderboard:  'RandomForest Classifier (Entropy)'
No recommended model yet.  Either autopilot is still running or models were trained manually.

Best model blueprint preprocessing steps:
['Ordinal encoding of categorical variables',
 'Category Count',
 'Missing Values Imputed',
 'Matrix of word-grams occurrences',
 'Elastic-Net Classifier (L2 / Binomial Deviance)',
 'RandomForest Classifier (Entropy)']

Best model metrics:
{'AUC': {'backtesting': None,
         'backtestingScores': None,
         'crossValidation': None,
         'holdout': None,
         'validation': 0.68726},
 'FVE Binomial': {'backtesting': None,
                  'backtestingScores': None,
                  'crossValidation': None,
                  'holdout': None,
                  'validation': 0.07861},
 'Gini Norm': {'backtesting': None,
               'backtestingScores': None,
               'crossValidation': None,
               'holdout': None,

### Alternatively we can compare how each model performs.

In [18]:
# Generate a summary of all model performances and put it into a DataFrame
models = project.get_models()
    
val_scores = pd.DataFrame([{'model_type': model.model_type,
                       'blueprint info': model.blueprint,
                       'model_id': model.id,
                       'sample_pct': model.sample_pct,
                       'featurelist': model.featurelist_name,
                       'val_logloss': model.metrics['LogLoss']['validation'],
                       'cross_val_logloss': model.metrics['LogLoss']['crossValidation']}
#                            for model in flist_models if model.metrics['LogLoss'] is not None])
                       for model in models if model.metrics['LogLoss'] is not None])

modelframe = val_scores.sort_values(by='cross_val_logloss')
modelframe

,blueprint info,cross_val_logloss,featurelist,model_id,model_type,sample_pct,val_logloss
0,Blueprint(RandomForest Classifier (Entropy)),None,my feature list,5be0f73a083f4d6b3a62bde7,RandomForest Classifier (Entropy),64.0,0.60466
1,Blueprint(Light Gradient Boosting on ElasticNe...,None,my feature list,5be0f73b083f4d6b3a62bde9,Light Gradient Boosting on ElasticNet Predicti...,64.0,0.60523
2,Blueprint(Light Gradient Boosted Trees Classif...,None,my feature list,5be0f73b083f4d6b3a62bdee,Light Gradient Boosted Trees Classifier with E...,64.0,0.60611
3,Blueprint(ExtraTrees Classifier (Gini)),None,my feature list,5be0f73b083f4d6b3a62bdea,ExtraTrees Classifier (Gini),64.0,0.60683
4,Blueprint(Elastic-Net Classifier (L2 / Binomia...,None,my feature list,5be0f73b083f4d6b3a62bdeb,Elastic-Net Classifier (L2 / Binomial Deviance...,64.0,0.60722
5,Blueprint(Vowpal Wabbit Classifier),None,my feature list,5be0f73b083f4d6b3a62bde8,Vowpal Wabbit Classifier,64.0,0.60764
6,Blueprint(Eureqa Generalized Additive Model Cl...,None,my feature list,5be0f73b083f4d6b3a62bded,Eureqa Generalized Additive Model Classifier (...,64.0,0.60892
7,Blueprint(RandomForest Classifier (Entropy)),None,my feature list,5be0f62b083f4d77851eaa76,RandomForest Classifier (Entropy),32.0,0.61277
8,Blueprint(Vowpal Wabbit Classifier),None,my feature list,5be0f62b083f4d77851eaa7a,Vowpal Wabbit Classifier,32.0,0.61280
9,Blueprint(Light Gradient Boosting on ElasticNe...,None,my feature list,5be0f62b083f4d77851eaa79,Light Gradient Boosting on ElasticNet Predicti...,32.0,0.61283


## Score new data
### Option 1 - direct on a project and model 
#### To make predctions on new data, simply upload a dataset to the project and request the predictions on the model you choose.  The scoring data should include all features except the target.

In [19]:
print('Scoring data on model \'%s\'\n' % best_model.model_type)

print('Uploading prediction dataset')
dataset_from_path = project.upload_dataset(df_pred)

print('Request predictions')
predict_job = best_model.request_predictions(dataset_from_path.id)

print('Waiting for prediction calculations')
predictions = predict_job.get_result_when_complete()

predictions.head()

Scoring data on model 'RandomForest Classifier (Entropy)'

Uploading prediction dataset
Request predictions
Waiting for prediction calculations


,positive_probability,prediction,prediction_threshold,row_id,class_0.0,class_1.0
0,0.546934,1.0,0.5,0,0.453066,0.546934
1,0.345241,0.0,0.5,1,0.654759,0.345241
2,0.664477,1.0,0.5,2,0.335523,0.664477
3,0.246766,0.0,0.5,3,0.753234,0.246766
4,0.505200,1.0,0.5,4,0.494800,0.505200


### Option 2 - Reference the Deployment ID
#### DataRobot's Model Management dashboard provides for monitoring model data including performance and datadrift.  Prediction requests are routed to a deployment and not a specific model.  This allows for swapping models in and out without requiring a code change.

#### First, from the DataRobot application: 
- 1) Go to the model
- 2) Click 'Predict'
- 3) Click 'Deploy Model API'
- 4) Click the orange 'Add New Deployment' button
- 5) On the Deployments page, for a given deploment, click 'Integrations'
- 6) Copy the DEPLOYMENT_ID and pass that to DataRobot API via the REST call

In [20]:
API_TOKEN = os.getenv('DATAROBOT_API_TOKEN')
ENDPOINT = os.getenv('DATAROBOT_ENDPOINT')
USERNAME = 'matthew.cohen@datarobot.com'

#

DEPLOYMENT_ID = '5be0e7a1fd2b980c56eecbc7'  # My project's recommended model: XGBoost @ 80%

pred_file = 'pred.csv'
df_pred.to_csv(pred_file)
data = open(pred_file, 'rb').read()  

headers = {'Content-Type': 'text/plain; charset=UTF-8', 'datarobot-key': '544ec55f-61bf-f6ee-0caf-15c7f919a45d'}
predictions_response = requests.post('https://cfds-ccm-prod.orm.datarobot.com/predApi/v1.0/deployments/%s/predictions' % (DEPLOYMENT_ID),
                                     auth=(USERNAME, API_TOKEN), data=data, headers=headers)

predictions_response.raise_for_status()
df = pd.DataFrame(predictions_response.json().get('data'))

# Flatten nested label/value dict to format for output
df['label1'] = None
df['proba1'] = None
df['label2'] = None
df['proba2'] = None
def func(row):
    for i, pair in enumerate(row['predictionValues']):
        name = pair.get('label')
        val = pair.get('value')
        col_name_n = 'label' + str(i+1)
        col_val_n = 'proba' + str(i+1)
        row[col_name_n] = name
        row[col_val_n] = val
    return row
df_pred_out = df.apply(lambda row: func(row), axis=1)
del df_pred_out['predictionValues']
print('pred out file shape:',df_pred_out.shape)

df_pred_out.head()

pred out file shape: (5000, 7)


,prediction,predictionThreshold,rowId,label1,proba1,label2,proba2
0,0.0,0.5,0,1.0,0.490129,0.0,0.509871
1,0.0,0.5,1,1.0,0.302685,0.0,0.697315
2,1.0,0.5,2,1.0,0.695894,0.0,0.304106
3,0.0,0.5,3,1.0,0.206731,0.0,0.793269
4,1.0,0.5,4,1.0,0.590266,0.0,0.409734
